In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

In [2]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42) # gpu vars
torch.backends.cudnn.deterministic = True  #needed
torch.backends.cudnn.benchmark = False

# TODO: 
- [ ] try to use only second layer's hidden state
- [ ] word embedding multiply tfidf
- [ ] use word2vec
- [ ] dropout to 0.4 and use adam to speedup, add dropout after cnn-relu

## Load and prepare data

In [3]:
train_file = 'data/train.csv'
val_file = 'example/val.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_seq': 40,
    'min_freq': 0,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [4]:
AttractiveData = AttractiveData(train_file, val_file, test_file, pretrained_file, config)

/home/yao/anaconda3/envs/DS_hw2/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/yao/anaconda3/envs/DS_hw2/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/yao/anaconda3/envs/DS_hw2/lib/python3.6/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be 

In [5]:
# for i, sentence in enumerate(AttractiveData.test_data):
#     if i == 3:
#         print(vars(AttractiveData.train_data[i]), vars(sentence))

In [6]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

38

## Start to train

In [7]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 8
config['hidden_dim'] = 30
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 200
config['lr'] = {
    'encoder': 1e-5,
    'embedding': 6e-6,
    'linear': 1e-5
}
config['num_layers'] = 1
config['kernel_size'] = 3
config['dropout'] = 0.3
config['train_len'] = AttractiveData.train_len
config['val_len'] = AttractiveData.val_len
config['test_len'] = AttractiveData.test_len

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12699, 300])


In [8]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, AttractiveData.valloader, pretrained_embeddings)

/home/yao/anaconda3/envs/DS_hw2/lib/python3.6/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [9]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(12699, 300, padding_idx=1)
   )
   (bigramcnn): Sequential(
     (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
     (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
     (3): ReLU()
     (4): Dropout(p=0.3, inplace=False)
   )
   (trigramcnn): Sequential(
     (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
     (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
     (3): ReLU()
     (4): Dropout(p=0.3, inplace=False)
   )
   (encoder_bigram_first): LSTM(100, 30, batch_first=True, dropout=0.3, bidirectional=True)
   (encoder_trigram_first): LSTM(100, 30, batch_first=True, dropout=0.3, bidirectional=True)
   (linear): Sequential(
     (0): Linear(in_features=124, out_features=30, bias=True)
     (1): ReLU()
     (2): Linear(in_features=30, out_features=1, bias=True)
   )
 ),
 4357441,
 4357441)

In [10]:
AttractiveTrainer.train()

Epoch:   0%|          | 0/200 [00:00<?, ?it/s]/home/yao/anaconda3/envs/DS_hw2/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
Epoch:   0%|          | 1/200 [00:00<01:16,  2.60it/s]


EP_0 | train loss: 3.6050523122151694 | val loss: 4.077105933544683 |


Epoch:   1%|          | 2/200 [00:00<01:14,  2.66it/s]


EP_1 | train loss: 0.5863744445875579 | val loss: 0.7456433889912624 |


Epoch:   2%|▏         | 3/200 [00:01<01:13,  2.69it/s]

...


EP_197 | train loss: 0.011294412554479112 | val loss: 0.01355458912896175 |


Epoch: 100%|█████████▉| 199/200 [01:13<00:00,  2.72it/s]


EP_198 | train loss: 0.013769246141115824 | val loss: 0.015283631854781918 |


Epoch: 100%|██████████| 200/200 [01:13<00:00,  2.71it/s]


EP_199 | train loss: 0.008749279306799758 | val loss: 0.0115039302584003 |


## for classification, not better

## Below is testing

In [10]:
from attractivenet import AttractiveNet

PATH = './model/CNN_LSTM_20201105-233735/0.371396.70'

load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(12699, 300, padding_idx=1)
  )
  (bigramcnn): Sequential(
    (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
  )
  (trigramcnn): Sequential(
    (0): Conv1d(300, 200, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(200, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
  )
  (encoder_bigram_first): LSTM(100, 30, batch_first=True, dropout=0.3, bidirectional=True)
  (encoder_trigram_first): LSTM(100, 30, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=124, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=1, bias=True)
  )
)

In [11]:
def predict_attractive(sentence, category, phase):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(0)
    # print(tensor_sentence.shape)

    prediction = load_model(tensor_sentence, tensor_category, phase=phase)

    return prediction

In [12]:
# train mean = 3.15, test mean = 2.8
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        prediction = predict_attractive(sentence.Headline, sentence.Category, 'test')
        predict_list.append(prediction.item())
        # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

## Below for statistics

In [13]:
import statistics
from sklearn.metrics import mean_squared_error

In [14]:
# # train mean = 3.15, test mean = 2.8
# train_list = []
# for i, sentence in enumerate(AttractiveData.train_data):
#     prediction = predict_attractive(sentence.Headline, sentence.Category, 'train')
#     train_list.append(prediction.item())
#     # train_list.append(prediction.item())
# # print(train_list)
# mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

In [15]:
# train_list[0:5], pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list()[0:5]

In [16]:
# a = AttractiveData.df_train['Label'].to_list()
# statistics.mean(a), statistics.stdev(a)

In [17]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8177459723098686, 0.35145273498457985)

In [18]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.01970666843872968, 2.832396388579045, 0.3599597701326745)

In [18]:
mean_squared_error(pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list(), pd.read_csv('./predict/ensemble.csv').sort_values(['ID']).Label.to_list())

0.1609754968529207

In [10]:
from sklearn.model_selection import KFold
import numpy as np

In [12]:
len(AttractiveData.train_data.examples)

2040